In [1]:
# Dependencies
import pandas as pd
import random
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import sem
import os
from pathlib import Path
import requests
import json
from scipy.stats import linregress, pearsonr
import scipy.stats as st


In [2]:
#load files
outbound_shipments_2016 = os.path.join(".", "resources", "outbound_shipments_2016.csv")
period_num = os.path.join(".", "resources", "period_numbers_2016to2019.csv")
states = os.path.join(".", "resources", "states.csv")

#read data
outbound_shipments_2016_df = pd.read_csv(outbound_shipments_2016)
period_num_df = pd.read_csv(period_num)
states_df = pd.read_csv(states)



In [3]:
#####2016 DATASET

#merge outbound_shipments_2018 and period_num_df
combined_2016_outbound_period_df = pd.merge(
    outbound_shipments_2016_df,
    period_num_df[["date", "period_number"]],  
    left_on="Shipped Date",
    right_on="date",
    how="left").drop(columns="date")

#combined_outbound_period_df with states_df
combined_2016_dataset_df = pd.merge(
    combined_2016_outbound_period_df,
    states_df[["state_abr", "country"]],  
    left_on="Shipping State",
    right_on="state_abr",
    how="left").drop(columns="state_abr")

combined_2016_dataset_df

,Navision Order ID,Navision Created Date,Channel,Channel Order ID,Shipping Agent,Tracking No,Order Status,Status Summary,Shipping State,Shipped Date,Delivered On,Status On,Delivered,period_number,country
0,SYMSP107727,1/8/16 9:09,Mars Veterinary,NaN,FEDEX,607414896680,Shipped,Delivered,NC,1/8/16,2/4/16,2/4/16 12:31,1.0,1,US
1,SYMSP107544,1/6/16 15:33,WisdomPanel.com,10069422,USPS,9400111899563326366253,Shipped,"Delivered, In/At Mailbox",MD,1/7/16,1/28/16,1/28/16 12:28,1.0,1,US
2,SYMSP108323,1/13/16 13:10,WisdomPanel.com,10069884,USPS,9400111899563306796834,Shipped,Delivered,CA,1/14/16,2/1/16,2/1/16 15:06,1.0,1,US
3,SYMSP108909,1/19/16 10:44,ROYAL CANIN USA,30286535,USPS,9400111899563389027979,Shipped,"Delivered, In/At Mailbox",NY,1/19/16,2/1/16,2/1/16 13:01,1.0,1,US
4,SYMSP108391,1/13/16 18:34,WisdomPanel.com,10069933,USPS,9400111899563306147803,Shipped,"Delivered, In/At Mailbox",MD,1/14/16,1/26/16,1/26/16 9:44,1.0,1,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47324,SYMSP151878,12/26/16 0:00,WisdomPanel.com,10109374,USPS,CJ457848965US,Shipped,Out,AB,12/27/16,NaN,1/4/17 0:00,0.0,13,CAN
47325,SYMSP151410,12/22/16 0:00,WisdomPanel.com,10108948,USPS,CJ457847973US,Shipped,Attempted,ON,12/27/16,NaN,1/3/17 0:00,0.0,13,CAN
47326,SYMSP151845,12/26/16 0:00,WisdomPanel.com,10109347,USPS,CJ457848744US,Shipped,Out,AB,12/27/16,NaN,1/4/17 0:00,0.0,13,CAN
47327,SYMSP151930,12/27/16 0:00,WisdomPanel.com,10109422,USPS,CJ457884636US,Shipped,Attempted,ON,12/27/16,NaN,1/4/17 0:00,0.0,13,CAN


In [4]:

combined_2016_dataset_df.dtypes


Navision Order ID         object
Navision Created Date     object
Channel                   object
Channel Order ID          object
Shipping Agent            object
Tracking No               object
Order Status              object
Status Summary            object
Shipping State            object
Shipped Date              object
Delivered On              object
Status On                 object
Delivered                float64
period_number              int64
country                   object
dtype: object

In [5]:
combined_2016_dataset_df

,Navision Order ID,Navision Created Date,Channel,Channel Order ID,Shipping Agent,Tracking No,Order Status,Status Summary,Shipping State,Shipped Date,Delivered On,Status On,Delivered,period_number,country
0,SYMSP107727,1/8/16 9:09,Mars Veterinary,NaN,FEDEX,607414896680,Shipped,Delivered,NC,1/8/16,2/4/16,2/4/16 12:31,1.0,1,US
1,SYMSP107544,1/6/16 15:33,WisdomPanel.com,10069422,USPS,9400111899563326366253,Shipped,"Delivered, In/At Mailbox",MD,1/7/16,1/28/16,1/28/16 12:28,1.0,1,US
2,SYMSP108323,1/13/16 13:10,WisdomPanel.com,10069884,USPS,9400111899563306796834,Shipped,Delivered,CA,1/14/16,2/1/16,2/1/16 15:06,1.0,1,US
3,SYMSP108909,1/19/16 10:44,ROYAL CANIN USA,30286535,USPS,9400111899563389027979,Shipped,"Delivered, In/At Mailbox",NY,1/19/16,2/1/16,2/1/16 13:01,1.0,1,US
4,SYMSP108391,1/13/16 18:34,WisdomPanel.com,10069933,USPS,9400111899563306147803,Shipped,"Delivered, In/At Mailbox",MD,1/14/16,1/26/16,1/26/16 9:44,1.0,1,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47324,SYMSP151878,12/26/16 0:00,WisdomPanel.com,10109374,USPS,CJ457848965US,Shipped,Out,AB,12/27/16,NaN,1/4/17 0:00,0.0,13,CAN
47325,SYMSP151410,12/22/16 0:00,WisdomPanel.com,10108948,USPS,CJ457847973US,Shipped,Attempted,ON,12/27/16,NaN,1/3/17 0:00,0.0,13,CAN
47326,SYMSP151845,12/26/16 0:00,WisdomPanel.com,10109347,USPS,CJ457848744US,Shipped,Out,AB,12/27/16,NaN,1/4/17 0:00,0.0,13,CAN
47327,SYMSP151930,12/27/16 0:00,WisdomPanel.com,10109422,USPS,CJ457884636US,Shipped,Attempted,ON,12/27/16,NaN,1/4/17 0:00,0.0,13,CAN


In [6]:
#calcualte transit time for 2016


#####2016 DATASET

#update datetime format, errors="coerce" replaces error values with NaT
combined_2016_dataset_df["Shipped Date"] = pd.to_datetime(combined_2016_dataset_df["Shipped Date"], errors="coerce")
combined_2016_dataset_df["Delivered On"] = pd.to_datetime(combined_2016_dataset_df["Delivered On"], errors="coerce")


#create "transit_time" column and calculate transit time 
combined_2016_dataset_df["transit_time"] = combined_2016_dataset_df.apply(
    lambda row: (row["Delivered On"] - row["Shipped Date"]).days if pd.notnull(row["Delivered On"]) and pd.notnull(row["Shipped Date"]) else None,
    axis=1)

#display df
combined_2016_dataset_df


/var/folders/0v/_cwy54bd71x52xsydh3mz7nh0000gn/T/ipykernel_81172/3908107113.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  combined_2016_dataset_df["Shipped Date"] = pd.to_datetime(combined_2016_dataset_df["Shipped Date"], errors="coerce")
/var/folders/0v/_cwy54bd71x52xsydh3mz7nh0000gn/T/ipykernel_81172/3908107113.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  combined_2016_dataset_df["Delivered On"] = pd.to_datetime(combined_2016_dataset_df["Delivered On"], errors="coerce")


,Navision Order ID,Navision Created Date,Channel,Channel Order ID,Shipping Agent,Tracking No,Order Status,Status Summary,Shipping State,Shipped Date,Delivered On,Status On,Delivered,period_number,country,transit_time
0,SYMSP107727,1/8/16 9:09,Mars Veterinary,NaN,FEDEX,607414896680,Shipped,Delivered,NC,2016-01-08,2016-02-04,2/4/16 12:31,1.0,1,US,27.0
1,SYMSP107544,1/6/16 15:33,WisdomPanel.com,10069422,USPS,9400111899563326366253,Shipped,"Delivered, In/At Mailbox",MD,2016-01-07,2016-01-28,1/28/16 12:28,1.0,1,US,21.0
2,SYMSP108323,1/13/16 13:10,WisdomPanel.com,10069884,USPS,9400111899563306796834,Shipped,Delivered,CA,2016-01-14,2016-02-01,2/1/16 15:06,1.0,1,US,18.0
3,SYMSP108909,1/19/16 10:44,ROYAL CANIN USA,30286535,USPS,9400111899563389027979,Shipped,"Delivered, In/At Mailbox",NY,2016-01-19,2016-02-01,2/1/16 13:01,1.0,1,US,13.0
4,SYMSP108391,1/13/16 18:34,WisdomPanel.com,10069933,USPS,9400111899563306147803,Shipped,"Delivered, In/At Mailbox",MD,2016-01-14,2016-01-26,1/26/16 9:44,1.0,1,US,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47324,SYMSP151878,12/26/16 0:00,WisdomPanel.com,10109374,USPS,CJ457848965US,Shipped,Out,AB,2016-12-27,NaT,1/4/17 0:00,0.0,13,CAN,NaN
47325,SYMSP151410,12/22/16 0:00,WisdomPanel.com,10108948,USPS,CJ457847973US,Shipped,Attempted,ON,2016-12-27,NaT,1/3/17 0:00,0.0,13,CAN,NaN
47326,SYMSP151845,12/26/16 0:00,WisdomPanel.com,10109347,USPS,CJ457848744US,Shipped,Out,AB,2016-12-27,NaT,1/4/17 0:00,0.0,13,CAN,NaN
47327,SYMSP151930,12/27/16 0:00,WisdomPanel.com,10109422,USPS,CJ457884636US,Shipped,Attempted,ON,2016-12-27,NaT,1/4/17 0:00,0.0,13,CAN,NaN


In [7]:
#display transit times by shipping agent, country, period_number, count 


#####2016 DATASET

#create new column "year" and adds  year based on "Shipped Date"
combined_2016_dataset_df["year"] = combined_2016_dataset_df["Shipped Date"].dt.year

# #filter shipping agents: FedEx and USPS
# filtered_df = combined_2017_dataset_df[
#     (combined_2017_dataset_df["Shipping Agent"].isin(["USPS", "FEDEX"])) &
#     (combined_2017_dataset_df["country"] == "US")]

#filter shipping agents: USPS and in the "country" = "US"
filtered_2016_df = combined_2016_dataset_df[
    (combined_2016_dataset_df["Shipping Agent"].isin(["USPS"])) &
    (combined_2016_dataset_df["country"] == "US")]

#group by "Shipping Agent", "country" ,"year", "period_number" 
transit_time_2016_stats_df = filtered_2016_df.groupby(["Shipping Agent", "country" ,"year", "period_number"]).agg(
    #calculate the avg_transit_time 
    avg_2016_transit_time=("transit_time", "mean"), 
    #copunt the number of times transit_time calculated 
    count=("transit_time", "size")).round(2).reset_index()


transit_time_2016_stats_df

,Shipping Agent,country,year,period_number,avg_2016_transit_time,count
0,USPS,US,2016,1,3.02,2742
1,USPS,US,2016,2,2.81,2478
2,USPS,US,2016,3,2.68,2272
3,USPS,US,2016,4,2.91,2241
4,USPS,US,2016,5,2.56,2286
5,USPS,US,2016,6,2.60,2407
6,USPS,US,2016,7,2.77,2699
7,USPS,US,2016,8,2.67,2527
8,USPS,US,2016,9,2.83,3410
9,USPS,US,2016,10,2.60,3038


In [8]:
#export the df to csv
transit_time_2016_stats_df.to_csv("output/usps_transit_time_2016.csv", index=False)

In [9]:
#display transit times by shipping agent, country, period_number, count 


#####2016 DATASET

#create new column "year" and adds  year based on "Shipped Date"
combined_2016_dataset_df["year"] = combined_2016_dataset_df["Shipped Date"].dt.year

# #filter shipping agents: FedEx and USPS
# filtered_df = combined_2016_dataset_df[
#     (combined_2016_dataset_df["Shipping Agent"].isin(["USPS", "FEDEX"])) &
#     (combined_2016_dataset_df["country"] == "US")]

#filter shipping agents: USPS and in the "country" = "US"
filtered_2017_df = combined_2016_dataset_df[
    (combined_2016_dataset_df["Shipping Agent"].isin(["FEDEX"])) &
    (combined_2016_dataset_df["country"] == "US")]

#group by "Shipping Agent", "country" ,"year", "period_number" 
fedex_transit_time_2016_stats = filtered_2016_df.groupby(["Shipping Agent", "country" ,"year", "period_number"]).agg(
    #calculate the avg_transit_time 
    avg_2016_transit_time=("transit_time", "mean"), 
    #copunt the number of times transit_time calculated 
    count=("transit_time", "size")).round(2).reset_index()


fedex_transit_time_2016_stats

,Shipping Agent,country,year,period_number,avg_2016_transit_time,count
0,USPS,US,2016,1,3.02,2742
1,USPS,US,2016,2,2.81,2478
2,USPS,US,2016,3,2.68,2272
3,USPS,US,2016,4,2.91,2241
4,USPS,US,2016,5,2.56,2286
5,USPS,US,2016,6,2.60,2407
6,USPS,US,2016,7,2.77,2699
7,USPS,US,2016,8,2.67,2527
8,USPS,US,2016,9,2.83,3410
9,USPS,US,2016,10,2.60,3038
